In [ ]:
%matplotlib inline
from IPython.display import clear_output
from pytesseract import image_to_string

from srbot import *

In [ ]:
#Crafting guild gold miner

gold_color = [251,204,31]

pottery_icon = load_image('pottery_icon.png')
mine_icon = load_image('mine_icon.png')

falador_wall = [189,175,152]
agility_color = [1,161,43]
tunnel_color = [60,9,0]
rimm_road = [117,109,108]

road_tol = 0.10

def mine_gold():
    print('searching for rocks')
    mainscreen = get_mainscreen()
    found = find_colors(gold_color,mainscreen,mode='hsl',tol=(0.08,0.15,0.15))
    if len(found) > 0:
        clusters,counts = cluster(found,10)
        found = np.asarray([np.mean(cluster.T,axis=1) for cluster in clusters])
        distsort = np.argsort(np.sum(np.square(found-[msw/2,msh/2-20]),axis=-1))
        found = found[distsort]
        for point in found[:min(100,len(found))]:
            move_mouse(*(point+[msxs,msys]))
            sleep(0.2)
            uptext = get_uptext(width=80)
            cyan = find_colors([0,238,238],uptext,tol=0.2)
            print('cyan',len(cyan))
            if len(cyan) > 100:
                inv = count_inv()
                click_mouse(*(point+[msxs,msys]))
                for i in range(20):
                    sleep(0.5)
                    if np.any(count_inv() != inv):
                        sleep(1.0)
                        return True
                return False
    if np.random.random() < 0.2:
        print('trying new spot')
        minimap = get_minimap()
        mine = find_best_bitmap(mine_icon,minimap,mode='xcorr',tol=0.2)
        if len(mine) > 0:
            np.random.shuffle(mine)
            click_mouse(*(mine[0]+[mmxs,mmys-15]))
            flag_wait()
    return False

def to_guild(rr,minmap):
    #check to see if we're at the guild
    minimap = get_minimap()
    a = find_best_bitmap(pottery_icon,minimap,mode='xcorr',tol=0.2)
    if len(a) > 1:
        guild_enterance = np.mean(a.T,axis=1)
        dist = np.sqrt(np.sum(np.square(guild_enterance-[mmxc-mmxs,mmyc-mmys])))
        if dist > 20: #walk to it
            print('approaching guild')
            click_mouse(*(guild_enterance+[mmxs,mmys]))
            flag_wait()
        else: #we're here!
            set_compass_angle(180,tol=7)
            print('looking for door')
            mainscreen = get_mainscreen()
            a,b = [43,33,8],[103,78,26]
            a = find_colors(a,mainscreen,tol=(1.0,0.08,0.08),mode='hsl')
            b = find_colors(b,mainscreen,tol=(1.0,0.08,0.08),mode='hsl')
            found = filter_near(a,b,20)
            print('candidates',len(found))
            if len(found) > 0:
                clusters,counts = cluster(found,radius=2)
                clusters = clusters[np.argsort(-counts)]
                for found in clusters[:min(5,len(clusters))]:
                    np.random.shuffle(found)
                    move_mouse(*(found[0]+[msxs,msys]))
                    sleep(0.1)
                    uptext = get_uptext(width=80)
                    cyan = find_colors([0,238,238],uptext,tol=0.2)
                    print('cyan',len(cyan))
                    if len(cyan) > 100:
                        click_mouse(*(found[0]+[msxs,msys]))
                        flag_wait()
                        if in_guild():
                            minimap = get_minimap()
                            mine = find_best_bitmap(mine_icon,minimap,mode='xcorr',tol=0.2)
                            if len(mine) > 0:
                                np.random.shuffle(mine)
                                click_mouse(*(mine[0]+[mmxs,mmys-15]))
                                flag_wait()
                            return True
            print('retrying guild approach')
            click_mouse(*(guild_enterance+[mmxs,mmys]))
            flag_wait()
    elif len(rr) == 0:
        print('pretty lost...')
    else:
        #otherwise... walk to guild
        order = np.argsort(-rr[:,1]+2*rr[:,0]) #sort by -y+2*x (sww-most)
        walkto = rr[order[0]]
        click_mouse(*(walkto+[mmxc,mmyc]))
        sleep(2.0)
    
def exit_guild(): 
    minimap = get_minimap()
    a = find_best_bitmap(pottery_icon,minimap,mode='xcorr',tol=0.2) 
    if len(a) > 0:
        guild_enterance = np.mean(a.T,axis=1)
        if np.sqrt(np.sum(np.square(guild_enterance-[mmxc-mmxs,mmyc-mmys]))) > 20:
            click_mouse(*(guild_enterance+[mmxs,mmys]))
            flag_wait()
        print('looking for door')
        mainscreen = get_mainscreen()
        a,b = [43,33,8],[103,78,26]
        a = find_colors(a,mainscreen,tol=(1.0,0.08,0.08),mode='hsl')
        b = find_colors(b,mainscreen,tol=(1.0,0.08,0.08),mode='hsl')
        found = filter_near(a,b,20)
        print('candidates',len(found))
        if len(found) > 0:
            clusters,counts = cluster(found,radius=2)
            found = clusters[np.argmax(counts)]
            np.random.shuffle(found)
            move_mouse(*(found[0]+[msxs,msys]))
            sleep(0.1)
            uptext = get_uptext(width=80)
            cyan = find_colors([0,238,238],uptext,tol=0.2)
            print('cyan',len(cyan))
            if len(cyan) > 100:
                click_mouse(*(found[0]+[msxs,msys]))
                run_on()
                flag_wait()
                return True
        click_mouse(*(guild_enterance+[mmxs,mmys]))
        flag_wait()
    else:
        click_mouse(mmxc+15,mmyc+30)
        flag_wait()

def in_guild():
    minimap = get_minimap()
    
    a = find_best_bitmap(pottery_icon,minimap,mode='xcorr',tol=0.2) 
    b = find_best_bitmap(mine_icon,minimap,mode='xcorr',tol=0.2)
    if len(a)+len(b) < 1:
        return False
        
    mainscreen = get_mainscreen()

    a,b = [98,83,43],[87,81,81]
    a = find_colors(a,mainscreen,tol=(0.03,0.08,0.2),mode='hsl')
    b = find_colors(b,mainscreen,tol=(1.0,0.07,0.08),mode='hsl')

    #anear = filter_near(a,[[msxc-msxs,msyc-msys]],50)
    #bnear = filter_near(b,[[msxc-msxs,msyc-msys]],50)
    print(len(a),len(b))#,len(anear),len(bnear))

    in_mine = len(a) > 40000 #and (len(anear)+len(bnear)) > 3000
    in_guild = len(b) > 35000
    return in_mine or in_guild
    
target()
click_mouse(mmxc,mmyc)
total_trips = 0
last_mine = mark_time()
logins = 0
while True:
    client = get_client()
    if len(find_bitmap(loginscreen,client)) > 0:
        logins = logins + 1
        if logins > 10:
            raise RuntimeError('Too many logins! Bailing out before we get fishy!')
        login()
        continue
    logins = 0
    
    if mark_time() - last_mine > 10*60:
        raise RuntimeError('No gold in over 10 minutes, ABORT!')
        
    if in_guild():
        set_compass_angle(180,tol=7,click=False)
        if count_inv() == 28:
            exit_guild()
        else:
            if mine_gold():
                last_mine = mark_time()
    else:
        set_compass_angle(0,tol=7,click=False)
        minimap = get_minimap()
        masked = np.full_like(minimap,0)
        masked[minimap_mask] = minimap[minimap_mask]
        minimap = masked
        inv_full = count_inv() == 28
        rr = find_colors(rimm_road,minimap,tol=road_tol,mode='hsl') - [mmxc-mmxs,mmyc-mmys]
        rr = filter_near(rr,[[0,0]],65)
        fw = find_colors(falador_wall,minimap,tol=0.15,mode='hsl') - [mmxc-mmxs,mmyc-mmys]
        npc_points = find_colors([255,255,0],minimap,tol=0.05,mode='hsl')
        bank_points = np.concatenate([find_colors(bank_floor,minimap,tol=0.085,mode='hsl') for bank_floor in bank_floor_colors])
        clusters,counts = cluster(bank_points)
        if len(counts) < 1 or np.count_nonzero(counts>20) < 1:
            bank_points = np.asarray([])
        else:
            bank_points = clusters[np.random.choice(np.nonzero(counts>20)[0])]
        bank_points = filter_near(npc_points,bank_points,6)
        if len(bank_points) < 20:
            agility = find_colors(agility_color,minimap,tol=(0.08,0.15,0.15),mode='hsl') - [mmxc-mmxs,mmyc-mmys]
        else:
            agility = []
        print('Locating...','R:',len(rr),'F:',len(fw),'B:',len(bank_points),'A:',len(agility))
        if len(fw) > 250: # inside falador either traveling or banking
            north_of_wall = None
            if len(agility) > 5 and len(filter_radius(agility,[0,0],30)) > 5: #close to hole
                near_wall = filter_radius(fw,[0,0],15)
                print('near wall',len(near_wall))
                if len(near_wall) > 5:
                    north_of_wall = np.mean(near_wall[:,1]) > 0 #mean of y position of wall within 30 px of character
            if north_of_wall is None:
                sw_wall_count = np.count_nonzero(np.logical_and(fw[:,0] < 0,fw[:,1] > 0))
                print('sw wall',sw_wall_count)
                north_of_wall = sw_wall_count > 15 #more than 20 wall px in s or w 
            print('north' if north_of_wall else 'south','of wall')
            if len(agility) > 5 and (north_of_wall ^ inv_full): #if we haven't gone under wall yet
                order = np.argsort(-2*fw[:,1]+fw[:,0]) #sort by -2*y+x (ssw-most)
                walkto = fw[order[0]]+[20,0]
                click_mouse(*(walkto+[mmxc,mmyc]))
                flag_wait()
                mainscreen = get_mainscreen()
                tunnel = find_colors(tunnel_color,mainscreen,mode='hsl',tol=0.08)
                if len(tunnel)>20:
                    np.random.shuffle(tunnel)
                    click = tunnel[0]
                    if north_of_wall:
                        click = tunnel[0]-[0,75]
                    click_mouse(*(click+[msxs,msys]))
                    if north_of_wall:
                        run_on()
                    sleep(8.0)
                    continue
            if inv_full: #definitely going north
                if open_bank(): #try to go to and open bank
                    deposit_all()
                    total_trips = total_trips + 1
                    clear_output()
                    print('Completed %i inventories'%total_trips)
                    continue    
                if len(bank_points) < 40:
                    if north_of_wall: #otherwise if inside falador go north
                        order = np.argsort(2*fw[:,1]+fw[:,0]) #sort by 2*y+x (nnw-most)
                        walkto = fw[order[0]]+[20,0]
                        click_mouse(*(walkto+[mmxc,mmyc]))
                        sleep(2.0)
                    else: #gotta get to hole
                        order = np.argsort(-2*fw[:,1]+fw[:,0]) #sort by -2*y+x (ssw-most)
                        walkto = fw[order[0]]+[20,0]
                        click_mouse(*(walkto+[mmxc,mmyc]))
                        flag_wait()
                else:
                    print('must have missed bank...')
            elif north_of_wall: # inside falador, north of wall, and inv empty: go south
                order = np.argsort(-2*fw[:,1]+fw[:,0]) #sort by -2*y+x (ssw-most)
                walkto = fw[order[0]]+[20,0]
                click_mouse(*(walkto+[mmxc,mmyc]))
                sleep(2.0)
            else: #gotta get to guild
                to_guild(rr,minimap)
        elif inv_full: # full inventory travelin to bank
            if len(agility) > 5:
                np.random.shuffle(agility)
                walkto = agility[0]
            elif len(rr) > 0:
                order = np.argsort(rr[:,1]+rr[:,0]) #sort by y+x (nw-most)
                walkto = rr[order[0]]
            else:
                walkto = np.asarray([-20,-40])
            click_mouse(*(walkto+[mmxc,mmyc]))
            sleep(2.0)
        else: #mining or traveling to mine
            to_guild(rr,minimap)